In [2]:
!pip install scipy
!pip install pillow
!pip install pandas
!pip install tensorflow_addons

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 10.4MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 1.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 1.8MB 577kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [2]:
BATCH_SIZE = 64
IMAGE_SIZE = 224

In [3]:
df = pd.read_csv("data/train_labels.txt")

In [4]:
df['id'] = df['id'].apply(lambda x: f"{'0'* (6-len(str(x)))}{x}.png")
df['class'] = df['class'].apply(str)

In [5]:
df.head()

,id,class
0,000001.png,0
1,000002.png,0
2,000003.png,0
3,000004.png,0
4,000005.png,0


In [6]:
img_gen = ImageDataGenerator(validation_split=0.15)

In [7]:
@tf.function
def f1(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return 2 * (K.sum(y_true * y_pred)+ K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

In [8]:
def model():
    head = tf.keras.Sequential()
    head.add(layers.Conv2D(32, (3, 3), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    head.add(layers.BatchNormalization())
    head.add(layers.Activation('relu'))
    head.add(layers.MaxPooling2D(pool_size=(2, 2)))

    head.add(layers.Conv2D(32, (3, 3)))
    head.add(layers.BatchNormalization())
    head.add(layers.Activation('relu'))
    head.add(layers.MaxPooling2D(pool_size=(2, 2)))

    head.add(layers.Conv2D(64, (3, 3)))
    head.add(layers.BatchNormalization())
    head.add(layers.Activation('relu'))
    head.add(layers.MaxPooling2D(pool_size=(2, 2)))

    average_pool = tf.keras.Sequential()
    average_pool.add(layers.AveragePooling2D())
    average_pool.add(layers.Flatten())
    average_pool.add(layers.Dense(2, activation='softmax'))

    standard_model = tf.keras.Sequential([
        head,
        average_pool
    ])
    return standard_model

In [9]:
standard_model = model()

In [ ]:
standard_model.compile(optimizer=tf.keras.optimizers.Adam(),
                       loss='binary_crossentropy',
                       metrics=['accuracy', tfa.metrics.F1Score(num_classes=2, average=None)])
standard_model.fit(img_gen.flow_from_directory(directory='data/data/', 
                            class_mode='binary', batch_size=32, shuffle=True),epochs=7,
                   verbose=2)

Found 17000 images belonging to 2 classes.
